Customer Lifetime Value

In [1]:
import pandas as pd
import json

df = pd.read_json('raw_customer_data.json', lines=True, orient='records') # read the document store to dataframe
df = df.explode("transactions") # because trasnsactions is a list of dicts
id = df["id"].reset_index() # keep the ids 
df = pd.json_normalize(df["transactions"]) # dict to dataframe

data = pd.merge(id,df,left_index=True, right_index=True)

data

,index,id,settledAt,ftd,amount,status,type
0,0,128867,2022-03-10T14:14:34.583+0000,True,40.0,SUCCESS,deposit
1,1,128853,2022-03-10T11:30:45.292+0000,False,50.0,SUCCESS,deposit
2,1,128853,2022-03-10T09:55:57.469+0000,1.0,50.0,SUCCESS,deposit
3,2,128850,2022-03-10T11:06:32.510+0000,True,159.0,SUCCESS,deposit
4,2,128850,2022-03-10T11:32:38.318+0000,0.0,180.0,FAIL,deposit
...,...,...,...,...,...,...,...
180746,6675,5825,2020-03-23T08:26:33.212+0000,0.0,20.0,CANCEL,deposit
180747,6675,5825,2020-03-20T13:38:59.789+0000,0.0,25.0,FAIL,deposit
180748,6676,5774,2019-12-15T20:09:26.286+0000,False,50.0,ERROR,withdrawal
180749,6676,5774,2019-12-16T06:49:49.786+0000,0.0,50.0,SUCCESS,withdrawal


# EDA

In [2]:
#Get overview of the data
def dataoveriew(df):
    print('Number of rows: ', df.shape[0])
    print("\nNumber of features:", df.shape[1])
    print("\nData Features:")
    print(df.columns.tolist())
    print("\nMissing values:", df.isnull().sum().values.sum())
    print("\nUnique values:")
    print(df.nunique())

dataoveriew(data)

Number of rows:  180751

Number of features: 7

Data Features:
['index', 'id', 'settledAt', 'ftd', 'amount', 'status', 'type']

Missing values: 0

Unique values:
index          6677
id             6661
settledAt    180663
ftd               2
amount         9400
status            5
type              3
dtype: int64


In [3]:
data.dtypes

index          int64
id             int64
settledAt     object
ftd           object
amount       float64
status        object
type          object
dtype: object

In [4]:
data.groupby('status')['id'].count()

status
CANCEL      20662
ERROR        6621
FAIL        27021
INITIAL         7
SUCCESS    126440
Name: id, dtype: int64

In [5]:
data.groupby('type')['id'].count()

type
deposit       92958
transfer       6583
withdrawal    81210
Name: id, dtype: int64

In [6]:
# some ids have more than one 1.0 ftd count. it cannot be true.

data['ftd'] = data['ftd'].astype(float) # convert to float to be able to count 
data.groupby('id')['ftd'].value_counts().to_frame()

ftd
id     ftd     
5774   0.0    2
       1.0    1
5825   0.0    8
       1.0    1
5830   0.0    2
...         ...
128850 0.0    2
       1.0    2
128853 0.0    2
       1.0    2
128867 1.0    2

[12985 rows x 1 columns]

In [7]:
data[data['id']== 128785] # there are dublicate lines 9 and 140899

,index,id,settledAt,ftd,amount,status,type
9,5,128785,2022-03-14T09:25:37.516+0000,1.0,40.0,SUCCESS,deposit
10,5,128785,2022-03-14T09:12:39.813+0000,0.0,40.0,INITIAL,deposit
140899,5593,128785,2022-03-14T09:25:37.516+0000,1.0,40.0,SUCCESS,deposit
140900,5593,128785,2022-03-14T10:27:07.348+0000,0.0,40.0,FAIL,deposit


In [8]:
data = data.drop(['index'],axis=1)

data = data.drop_duplicates(ignore_index = True)   # remove dubplicates

In [9]:
# disregard all transactions that took place before the date of the ftd

import datetime

ids = list(data['id'].unique())

for id in ids:
    first_date = data[data['id'] == id][data['ftd']==1.0]['settledAt']
    first_date = pd.to_datetime(first_date.values)
    for k in (data[data['id'] == id][data['ftd']==0.0]['settledAt']).values :
        if k < first_date:
            i = data[data['id'] == id][data['settledAt'] == k].index
            data = data.drop(i)

C:\Users\B590\AppData\Local\Temp\ipykernel_11212\3200623660.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  first_date = data[data['id'] == id][data['ftd']==1.0]['settledAt']
C:\Users\B590\AppData\Local\Temp\ipykernel_11212\3200623660.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for k in (data[data['id'] == id][data['ftd']==0.0]['settledAt']).values :
C:\Users\B590\AppData\Local\Temp\ipykernel_11212\3200623660.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  first_date = data[data['id'] == id][data['ftd']==1.0]['settledAt']
C:\Users\B590\AppData\Local\Temp\ipykernel_11212\3200623660.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for k in (data[data['id'] == id][data['ftd']==0.0]['settledAt']).values :
C:\Users\B590\AppData\Local\Temp\ipykernel_11212\3200623660.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index

In [10]:
data

,id,settledAt,ftd,amount,status,type
0,128867,2022-03-10T14:14:34.583+0000,1.0,40.0,SUCCESS,deposit
1,128853,2022-03-10T11:30:45.292+0000,0.0,50.0,SUCCESS,deposit
2,128853,2022-03-10T09:55:57.469+0000,1.0,50.0,SUCCESS,deposit
3,128850,2022-03-10T11:06:32.510+0000,1.0,159.0,SUCCESS,deposit
4,128850,2022-03-10T11:32:38.318+0000,0.0,180.0,FAIL,deposit
...,...,...,...,...,...,...
180654,5825,2020-03-23T08:48:32.463+0000,1.0,20.0,SUCCESS,deposit
180656,5825,2020-03-23T09:31:08.219+0000,0.0,20.0,FAIL,deposit
180661,5774,2019-12-15T20:09:26.286+0000,0.0,50.0,ERROR,withdrawal
180662,5774,2019-12-16T06:49:49.786+0000,0.0,50.0,SUCCESS,withdrawal


In [11]:
data.groupby('status')['id'].count() # almost none of the dropped lines had status as success 

status
CANCEL      14898
ERROR        6558
FAIL        24087
INITIAL         6
SUCCESS    126391
Name: id, dtype: int64

In [12]:
data.groupby('type')['id'].count() # almost all dropped lines where a deposit type

type
deposit       84182
transfer       6576
withdrawal    81182
Name: id, dtype: int64